In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import unicodedata

Steps:
1. Download data from sales force. Make sure to choose details only  and .xls format. 
2. Open in excel and save as a .csv file
3. Delete the "Current crossing method" column that contains the embeding codes <img...
4. Change the name of the input file to the name and path of the report
5. Change the name and path of the output file
6. Search for ? and replace all with '

## Data Clean up
This is code that cleans up data from salesforce and creates geojson for map

In [19]:
# Change Variables below
input_file = './home/data/report1679360399851.csv'
output_file = 'home/data/civ-assessments-v7.geojson'

In [106]:
gdf = gpd.read_file(input_file, encoding='iso-8859-1')
gdf['latitude'] = gdf['Bridge Opportunity: GPS (Latitude)']
gdf['longitude'] = gdf['Bridge Opportunity: GPS (Longitude)']
gdf['latitude'] = gdf['latitude'].astype(float)
gdf['longitude'] = gdf['longitude'].astype(float)
# set geometry to latitude and longitude columns
gdf['geometry'] = gdf.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)
gdf.columns = gdf.columns.str.strip()
gdf = gdf.drop(columns=['Current crossing method', "Site Validation Photo URL", 'High water line', 'High water line URL', 'Current crossing method URL', 'Current crossing method', "Site Validation Photo", "Rejection Image 1", "Rejection Image 2", "Current crossing method URL", "High water line URL", "Downstream URL", "Downstream", "Upstream URL", "Upstream", "Left top of bank (toward) URL", "Left top of bank (toward)", "Left top of bank (away) URL", "Left top of bank (away)", "Right top of bank (toward) URL", "Right top of bank (toward)", "Right top of bank (away) URL", "Right top of bank (away)", "Form: Created By"])
# remove all rows that have "Duplicate" in the "Bridge Opportunity: Sub-Stage" column
print ("Dataframe is " + str(len(gdf)) + ", rows in length")
gdf = gdf[gdf['Bridge Opportunity: Sub-Stage'] != 'Duplicate']
print ("Dataframe is " + str(len(gdf)) + " rows in length")
gdf.columns = gdf.columns.str.replace('Bridge Opportunity:', '')
gdf.columns = gdf.columns.str.strip()
# change type of values in 'Width of River During Flooding (m)' to number
gdf['Width of River During Flooding (m)'] = pd.to_numeric(gdf['Width of River During Flooding (m)'], errors='coerce')
gdf['Days per year river is flooded'] = pd.to_numeric(gdf['Days per year river is flooded'], errors='coerce')
gdf['River crossing deaths in last 3 years'] = pd.to_numeric(gdf['River crossing deaths in last 3 years'], errors='coerce')
# adding priority column
priority = [1042341, 1042379, 1042340, 1042608, 1042607, 1042582, 1042661, 1042662, 1042667, 1042668, 1042610, 1042827, 1044824, 1044813, 1040878, 1043642, 1044656, 1043204, 1043762, 1043765, 1044792, 1042768, 1043253, 1042444, 1042814, 1043168, 1043282, 1044242, 1044241, 1044234, 1044235, 1044236, 1044237, 1042617, 1042623, 1042675, 1042848, 1042681, 1043691, 1044028, 1044049]
priority = [str(i) for i in priority]
gdf['priority'] = np.where(gdf['Opportunity Unique Identifier'].isin(priority), 'yes', 'no')
print (gdf['priority'].value_counts())
gdf.to_csv('home/data/civ-assessments-v7.csv', encoding='iso-8859-1')

Dataframe is 676, rows in length
Dataframe is 624 rows in length
no     584
yes     40
Name: priority, dtype: int64


In [108]:
gdf

,Opportunity Name,Opportunity Unique Identifier,Bridge Name,Country,Level 3 Government,Level 4 Government,Stage,Sub-Stage,Individuals Directly Served,River Name,...,Waterway or Obstacle Classification,Transportation Route Classification,Environment Classification,Form: Created Date,Level 1 Government,Level 2 Government,geometry,latitude,longitude,priority
0,CÃ´te d'Ivoire - Adamadougou - 1043158,1043158,Adamadougou,CÃ´te d'Ivoire,Dogbo,Bledie,Identified,,,Carrefour adamadougou,...,"mudflat (low point with stagnant water, season...",motorcycle accessible path,Rural,11/11/22,San Pedro,San-PÃ©dro,POINT (-6.97600 4.85292),4.852916,-6.975996,no
1,CÃ´te d'Ivoire - Grobonou dan - 1042658,1042658,Grobonou dan,CÃ´te d'Ivoire,Gabiadji,Grobonou dan,Identified,,,Mnlin rene,...,River,Walking path,Rural,10/31/22,San Pedro,San-PÃ©dro,POINT (-6.40658 5.05259),5.052587,-6.406580,no
2,CÃ´te d'Ivoire - Sarakakro - 1042430,1042430,Sarakakro,CÃ´te d'Ivoire,Gabiadji,Gnity,Rejected,Vehicular Bridge Preferred,,Menegbe,...,"mudflat (low point with stagnant water, season...",motorcycle accessible path,Rural,10/20/22,San Pedro,San-PÃ©dro,POINT (-6.51919 5.17674),5.176743,-6.519194,no
3,CÃ´te d'Ivoire - Pont souleman - 1042372,1042372,Pont souleman,CÃ´te d'Ivoire,Gabiadji,Fahe,Identified,,,RiviÃ¨re souleman,...,"mudflat (low point with stagnant water, season...",non-graded road or car/vehicle accessible path,Rural,10/20/22,San Pedro,San-PÃ©dro,POINT (-6.66006 4.97557),4.975573,-6.660057,no
5,CÃ´te d'Ivoire - Pont ladji - 1042376,1042376,Pont ladji,CÃ´te d'Ivoire,Gabiadji,Fahe,Identified,,,Bas-fonds ladji,...,swamp (water year round),motorcycle accessible path,Rural,10/20/22,San Pedro,San-PÃ©dro,POINT (-6.66497 4.97133),4.971329,-6.664966,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,CÃ´te d'Ivoire - Appiakro 1 - 1043494,1043494,Appiakro 1,CÃ´te d'Ivoire,Dibri Assirikro,Angokoun-Kpangbassou,Identified,,,M?bandama 1,...,River; swamp (water year round),Walking path,Rural,11/27/22,GbÃªkÃª,Sakassou,POINT (-5.25535 7.61773),7.617728,-5.255355,no
670,CÃ´te d'Ivoire - SanguiÃ©kro 3 - 1043198,1043198,SanguiÃ©kro 3,CÃ´te d'Ivoire,Dibri Assirikro,Angokoun-Kpangbassou,Identified,,,Yourou 2,...,River,Walking path,Rural,11/13/22,GbÃªkÃª,Sakassou,POINT (-5.20230 7.62020),7.620202,-5.202297,no
671,CÃ´te d'Ivoire - SanguiÃ©kro 1 - 1043196,1043196,SanguiÃ©kro 1,CÃ´te d'Ivoire,Djibri Assirikro,Angokoun-Kpangbassou,Identified,,,Yourou 1,...,River,Walking path,Rural,11/13/22,GbÃªkÃª,Sakassou,POINT (-5.21483 7.61724),7.617245,-5.214832,no
672,CÃ´te d'Ivoire - Golibo 4 - 1043306,1043306,Golibo 4,CÃ´te d'Ivoire,Dibri Assirikro,Angokoun-Kpangbassou,Identified,,,Loka 2,...,River,Walking path,Rural,11/20/22,GbÃªkÃª,Sakassou,POINT (-5.27464 7.61565),7.615648,-5.274645,no


## This code is used to create unique clustering

In [109]:
# convert "River crossing deaths in last 3 years" to int
filter_gdf = gdf
filter_gdf['River crossing deaths in last 3 years'] = filter_gdf['River crossing deaths in last 3 years'].astype(int)
# create a dataframe with rows where "River crossing deaths in last 3 years" is greater than 0
mgdf = filter_gdf[filter_gdf['River crossing deaths in last 3 years'] > 0]
# create a dataframe with rows where "Flag for Rejection" No
ngdf = filter_gdf[filter_gdf['Flag for Rejection'] == 'No']
# create a dataframe from mgdf where "Flag for Rejection" is No
mngdf = mgdf[mgdf['Flag for Rejection'] == 'No']
# write mgdf, ngdf, and mn gdf to geojson files
mgdf.to_file(output_file[:-8]+'-mgdf.geojson', driver='GeoJSON')
ngdf.to_file(output_file[:-8]+'-ngdf.geojson', driver='GeoJSON')
mngdf.to_file(output_file[:-8]+'-mngdf.geojson', driver='GeoJSON')

## Exports all data to shape files

In [ ]:
import os
for files in os.listdir('./home/data/datawork'):
    # type is the last 8 characters of files
    filetype = files[-8:]
    filename = files[:-8]
    print (files)
    # read file as geodataframe
    if filetype == '.geojson':
        tempgdf = gpd.read_file('./home/data/datawork/' + files, encoding='iso-8859-1')        
        # write to csv
        tempgdf.to_csv('./home/data/outputs/' + filename + '.csv')
        # write to shp file
        tempgdf.to_file('./home/data/outputs/' + filename + '.shp', driver='ESRI Shapefile')

## Creates a vector population file from geotiff

In [53]:
x = gpd.read_file("./home/data/poly_grid.geojson", )

In [56]:
x

,id,left,top,right,bottom,geometry
0,1,-8.599583,10.73625,-8.589583,10.72625,"POLYGON ((-8.59958 10.73625, -8.58958 10.73625..."
1,2,-8.599583,10.72625,-8.589583,10.71625,"POLYGON ((-8.59958 10.72625, -8.58958 10.72625..."
2,3,-8.599583,10.71625,-8.589583,10.70625,"POLYGON ((-8.59958 10.71625, -8.58958 10.71625..."
3,4,-8.599583,10.70625,-8.589583,10.69625,"POLYGON ((-8.59958 10.70625, -8.58958 10.70625..."
4,5,-8.599583,10.69625,-8.589583,10.68625,"POLYGON ((-8.59958 10.69625, -8.58958 10.69625..."
...,...,...,...,...,...,...
389813,389814,-2.499583,4.40625,-2.489583,4.39625,"POLYGON ((-2.49958 4.40625, -2.48958 4.40625, ..."
389814,389815,-2.499583,4.39625,-2.489583,4.38625,"POLYGON ((-2.49958 4.39625, -2.48958 4.39625, ..."
389815,389816,-2.499583,4.38625,-2.489583,4.37625,"POLYGON ((-2.49958 4.38625, -2.48958 4.38625, ..."
389816,389817,-2.499583,4.37625,-2.489583,4.36625,"POLYGON ((-2.49958 4.37625, -2.48958 4.37625, ..."


In [55]:
import rasterstats as rs
import json

# Load GeoTIFF and GeoJSON data
tif_path = './home/data/civ_ppp_2020_constrained.tif'
json_path = './home/data/poly_grid.geojson'

# Calculate the zonal histogram
stats = rs.zonal_stats(json_path, tif_path, geojson_out=True, stats=['sum'])
zs = gpd.GeoDataFrame.from_features(stats)
zs.to_file('./home/data/civ-population-v2.geojson', driver='GeoJSON')


In [58]:
# read stats as geodataframe


In [59]:
zs

,geometry,id,left,top,right,bottom,sum
0,"POLYGON ((-8.59958 10.73625, -8.58958 10.73625...",1,-8.599583,10.73625,-8.589583,10.72625,NaN
1,"POLYGON ((-8.59958 10.72625, -8.58958 10.72625...",2,-8.599583,10.72625,-8.589583,10.71625,NaN
2,"POLYGON ((-8.59958 10.71625, -8.58958 10.71625...",3,-8.599583,10.71625,-8.589583,10.70625,NaN
3,"POLYGON ((-8.59958 10.70625, -8.58958 10.70625...",4,-8.599583,10.70625,-8.589583,10.69625,NaN
4,"POLYGON ((-8.59958 10.69625, -8.58958 10.69625...",5,-8.599583,10.69625,-8.589583,10.68625,NaN
...,...,...,...,...,...,...,...
389813,"POLYGON ((-2.49958 4.40625, -2.48958 4.40625, ...",389814,-2.499583,4.40625,-2.489583,4.39625,NaN
389814,"POLYGON ((-2.49958 4.39625, -2.48958 4.39625, ...",389815,-2.499583,4.39625,-2.489583,4.38625,NaN
389815,"POLYGON ((-2.49958 4.38625, -2.48958 4.38625, ...",389816,-2.499583,4.38625,-2.489583,4.37625,NaN
389816,"POLYGON ((-2.49958 4.37625, -2.48958 4.37625, ...",389817,-2.499583,4.37625,-2.489583,4.36625,NaN


In [62]:
# write zs to geojson
zs.to_file('./home/data/civ-population-v2.geojson', driver='GeoJSON')